# Optimizers

In the last few lessons, we learned how to build several neural network architectures.  We used gradient descent, a technique we originally learned in the [dense neural network](https://github.com/VikParuchuri/zero_to_gpt/blob/master/explanations/dense.ipynb) lesson, to adjust model parameters.

Gradient descent is a type of optimizer.  Optimizers adjust neural network parameters to try to get loss to (hopefully) a global minimum value.  In this lesson, we'll learn more about optimizers.  We'll first go into more depth on gradient descent, and discuss batch size, learning rate schedules, weight decay, and momentum.  We'll then discuss the AdamW optimizer, which is a popular optimizer that combines momentum and weight decay.  AdamW is the most commonly used optimizer in large language models like GPT.

## Stochastic Gradient Descent

The type of optimizer we've used so far is called stochastic gradient descent, or SGD.  In SGD, we take a minibatch of training examples, then compute the average gradient across the minibatch.  We then adjust the parameters using this average gradient.

Let's define a three-layer dense neural network, then explore the effect of batch size on our loss over time.  We'll first load in a dataset of house prices.

Each row in this dataset represents a single house.  The predictor columns are:

- `interest`: The interest rate
- `vacancy`: The vacancy rate
- `cpi`: The consumer price index
- `price`: The price of a house
- `value`: The value of a house
- `adj_price`: The price of a house, adjusted for inflation
- `adj_value`: The value of a house, adjusted for inflation

The predictor columns have all been scaled using the scikit-learn `StandardScaler`.  This gives each column a mean of 0 and a standard deviation of 1.  This makes it easier to activate our nonlinearities and have the network learn.

The target column is `next_quarter`, which is the price of the house in three months.  `next_quarter` has been scaled so the minimum value is `0`, and it has been divided by `1000` and rounded to the nearest integer.

In [22]:
import sys, os
sys.path.append(os.path.abspath('../data'))
from csv_data import HousePricesDatasetWrapper

wrapper = HousePricesDatasetWrapper()
train_data, valid_data, test_data = wrapper.get_flat_datasets()

In [23]:
# Show the train predictors
train_data[0][:2]

array([[ 1.94509048,  1.39642758, -1.52275521, -0.64519476, -0.11676266,
        -0.13891486,  0.8225858 ],
       [ 1.93249288,  1.39642758, -1.49354416, -0.64519476, -0.11676266,
        -0.15597175,  0.80219322]])

In [24]:
# Show the train target
train_data[1][:2]

array([[22.],
       [22.]])

Next, we'll define a single layer of our neural network.  We'll make a few modifications from earlier lessons:

- Instead of directly updating the parameters, we'll return the weight and bias gradients
- We'll add an update method that updates the weights later

This will enable us to swap different optimizers in and out of our network.

In [25]:
import math
import numpy as np

class Dense():
    def __init__(self, input_size, output_size, activation=True, seed=0):
        self.add_activation = activation
        self.hidden = None
        self.prev_hidden = None

        # Initialize the weights.  They'll be in the range -sqrt(k) to sqrt(k), where k = 1 / input_size
        np.random.seed(seed)
        k = math.sqrt(1 / input_size)
        self.weights = np.random.rand(input_size, output_size) * (2 * k) - k

        # Our bias will be initialized to 1
        self.bias = np.ones((1,output_size))

    def forward(self, x):
        # Copy the layer input for backprop
        self.prev_hidden = x.copy()
        # Multiply the input by the weights, then add the bias
        x = np.matmul(x, self.weights) + self.bias
        # Apply the activation function
        if self.add_activation:
            x = np.maximum(x, 0)
        # Copy the layer output for backprop
        self.hidden = x.copy()
        return x

    def backward(self, grad):
        # "Undo" the activation function if it was added
        if self.add_activation:
            grad = np.multiply(grad, np.heaviside(self.hidden, 0))

        # Calculate the parameter gradients
        w_grad = self.prev_hidden.T @ grad # This is not averaged across the batch, due to the way matrix multiplication sums
        b_grad = np.mean(grad, axis=0) # This is averaged across the batch
        param_grads = [w_grad, b_grad]

        # Calculate the next layer gradient
        grad = grad @ self.weights.T
        return param_grads, grad

    def update(self, w_grad, b_grad):
        # Update the weights given an update matrix
        self.weights += w_grad
        self.bias += b_grad

We can then use the `Dense` class to create a 3-layer neural network.  The first layer will take in our predictors and generate `25` hidden features, the second layer combine those features into `10` new features, and the final layer will make a prediction.

In [26]:
layers = [
    Dense(7, 25),
    Dense(25, 10),
    Dense(10, 1, activation=False)
]

We'll define functions that run forward and backward passes across all the layers together.  `forward` will do a full forward pass across all 3 layers, and `backward` will do a full backward pass across all 3 layers.

The backward pass will return the gradients instead of updating the parameters.  We'll use these gradients to update the parameters in our optimizer.

In [27]:
def forward(x, layers):
    # Loop through each layer
    for layer in layers:
        # Run the forward pass
        x = layer.forward(x)
    return x

def backward(grad, layers):
    # Save the gradients for each layer
    layer_grads = []
    # Loop through each layer in reverse order (starting from the output layer)
    for layer in reversed(layers):
        # Get the parameter gradients and the next layer gradient
        param_grads, grad = layer.backward(grad)
        layer_grads.append(param_grads)
    return layer_grads

We'll then define our SGD optimizer, which will take in a set of gradients, and use it to update the network parameters.

The process for SGD is:

- Normalize the gradient by batch size, so that the gradient is the average gradient across the batch (in our case, our bias gradient is already normalized, but the weight gradient is not)
- Multiply the gradient by learning rate
- Multiply the gradient by `-1` so it is subtracted from the parameters in the `update` function

In [28]:
class SGD():
    def __init__(self, lr):
        self.lr = lr

    def __call__(self, layer_grads, layers, batch_size):
        for layer_grad, layer in zip(layer_grads, reversed(layers)):
            w_grad, b_grad = layer_grad

            # Normalize the weight gradient by batch size
            w_grad /= batch_size

            # Calculate the update sizes
            w_update = -self.lr * w_grad
            b_update = -self.lr * b_grad

            layer.update(w_update, b_update)

## Monitoring

We can now write a function to train the network using the SGD optimizer and our data.  Since we'll be testing different batch sizes and optimizers, we want a way to monitor our network and compare different runs to wach other.  So far, we've been using print statements to monitor per-epoch accuracy in our network, which is hard to compare to other runs.

We'll use a tool called [Weights & Biases](https://wandb.ai) to monitor our network.  We can use it to track the loss and accuracy of our network, and compare different runs to each other.  W&B is a free tool, and you can sign up for an account [here](https://wandb.ai).  If you don't want to use W&B, you can also use TensorBoard, but it's harder to setup and use.

We'll start by importing `wandb` and logging in.  We'll also set the `WANDB_SILENT` environment variable to `True`, so that W&B doesn't print out a lot of system messages.

In [29]:
%env WANDB_SILENT=True

import wandb
wandb.login()

env: WANDB_SILENT=True


True

With W&B, we track each training run separately.  W&B will track the loss in each epoch, or anything else we want to keep track of.  It will also render graphs for us, so we can compare different runs against each other.

The key W&B functions are:

- `wandb.init` - This will initialize a new run.  We can use the `config` parameter to pass in run-specific information we want to view.
- `wandb.log` - This will log a dictionary of metrics to the current run.
- `wandb.define_metric` - This will define a metric that we want to track.  You normally don't need to define wandb metrics upfront, but we'll use a custom `step_metric` to ensure that results from runs with different batch sizes line up.

We'll use W&B to log a running training loss, the final training set loss each epoch, and the validation set loss.  We'll also track how long each epoch takes to run.

## The Training Loop

We can now write a function that will train our network using SGD, and log the loss to W&B.  This function will be very similar to training loops that we've written in previous lessons.

We won't do this here, but you usually want to shuffle your training data each epoch to ensure that batch composition changes.  Random shuffling is important when our batch size is greater than `1`.  As we saw earlier, the gradient is averaged across all the examples in a batch.  Random shuffling will put different training examples together each time, allowing the model to see more combinations of gradients.  This reduces overfitting, and improves the generalization of our network.  It can also make SGD converge faster (get to the global minimum error).

In our case, we won't do random shuffling, since the data is time series.  Shuffling can actually cause validation error to increase significantly with time series data.

In [30]:
import time

def training_run(epochs, batch_size, optimizer, train_data, valid_data):
    # Initialize a new W&B run, with the right parameters
    wandb.init(project="optimizers",
               config={"batch_size": batch_size,
                       "lr": optimizer.lr,
                       "epochs": epochs,
                       "optimizer": type(optimizer).__name__})

    # Setup the metrics we want to track with wandb
    wandb.define_metric("batch_step") # This will ensure that results from runs with different batch sizes line up
    wandb.define_metric("epoch") # This will ensure that results from runs with different batch sizes line up
    wandb.define_metric("valid_loss", step_metric="epoch")
    wandb.define_metric("train_loss", step_metric="epoch")
    wandb.define_metric("runtime", step_metric="epoch")
    wandb.define_metric("running_loss", step_metric="batch_step")

    # Setup the layers for the training run
    layers = [
        Dense(7, 25),
        Dense(25,10),
        Dense(10, 1, activation=False)
    ]

    # Split the training and valid data into x and y
    train_x, train_y = train_data
    valid_x, valid_y = valid_data

    for epoch in range(epochs):
        running_loss = 0
        start = time.time() # The start time of our run

        for i in range(0, len(train_x), batch_size):
            # Get the x and y batches
            x_batch = train_x[i:(i+batch_size)]
            y_batch = train_y[i:(i+batch_size)]
            # Make a prediction
            pred = forward(x_batch, layers)

            # Run the backward pass
            loss = pred - y_batch
            layer_grads = backward(loss, layers)

            # Run the optimizer
            optimizer(layer_grads, layers, batch_size)

            # Update running loss
            running_loss += np.mean(loss ** 2)

            batch_idx = i + batch_size # Get the last index of the current batch
            batch_step = batch_idx + epoch * len(train_x)
            # Log running loss.  We multiply by batch size to offset the mean from earlier.
            wandb.log({"running_loss": running_loss / batch_idx * batch_size, "batch_step": batch_step})

        # Calculate and log validation loss
        valid_preds = forward(valid_x, layers)
        valid_loss = np.mean((valid_preds - valid_y) ** 2)
        train_loss = running_loss / len(train_x) * batch_size
        wandb.log({
            "valid_loss": valid_loss,
            "epoch": epoch,
            "train_loss": train_loss,
            "runtime": time.time() - start
        })

    # Mark the run as complete
    wandb.finish()

We can then initialize the parameters we want to adjust (like batch size), and start the run.

In [31]:
# Setup our parameters
epochs = 100
batch_size = 4
lr = 1e-4

# Run our training loop
sgd = SGD(lr=lr)
training_run(epochs, batch_size, sgd, train_data, valid_data)

You should now be able to see your run in W&B.  You will need to click on the `optimizers` project in your [dashboard](https://wandb.ai/home).

## Batch Size

We can now compare our first run to a run with a higher batch size:

In [32]:
# Setup our parameters
epochs = 100
batch_size = 32
lr = 1e-4

# Run our training loop
sgd = SGD(lr=lr)
training_run(epochs, batch_size, sgd, train_data, valid_data)

You should end up with a dashboard that looks like this:

![W&B Dashboard](images/optimizers/wandb_comp2.png)

We can draw a few conclusions from this:

1. The higher batch size descends more slowly that a lower batch size
2. The higher batch size converges to a lower validation loss, but a higher training loss
3. The higher batch size runs much faster (around 8x faster)

Batch size can make a big impact on the final accuracy of your model.  In general, a higher batch size is a form of regularization (which we'll talk about in depth in a later lesson).  Regularization increases training loss, but decreases validation loss.  This happens because you're averaging over multiple training examples in the batch, and making it harder to overfit to specific examples.

Higher batch sizes can also run much faster than lower batch sizes.  In practice, if the batch size is too high, then validation loss starts to suffer.  So you need to find the batch size that runs quickly and minimizes validation loss.  This is different for each dataset, so it can take some experimentation.

## Momentum

One downside to the basic SGD optimizer is that it has no knowledge of past gradients.  It has to make the decision on how to adjust the parameters based only on the current gradient.  It's like asking you to predict if a stock price will go up or down tomorrow.  If you only know how the stock did today, your guess will be worse than if you know how the stock did over the past year.

We can solve this with momentum.  With momentum, we give the optimizer knowledge of the direction the gradient is moving in.

In [42]:
class SGDMomentum():
    def __init__(self, lr, alpha):
        self.lr = lr
        self.alpha = alpha
        self.velocities = None

    def initialize_velocities(self, layer_grads):
        self.velocities = []
        for layer_grad in layer_grads:
            w_grad, b_grad = layer_grad
            initial_velocities = [np.zeros_like(w_grad), np.zeros_like(b_grad)]
            self.velocities.append(initial_velocities)

    def __call__(self, layer_grads, layers, batch_size):
        if self.velocities is None:
            self.initialize_velocities(layer_grads)

        new_velocities = []
        for layer_grad, velocity, layer in zip(layer_grads, self.velocities, reversed(layers)):
            w_grad, b_grad = layer_grad
            w_vel, b_vel = velocity

            # Normalize the weight gradient by batch size
            w_grad /= batch_size

            # Calculate the update sizes
            w_vel = w_vel * self.alpha - self.lr * w_grad
            b_vel = b_vel * self.alpha - self.lr * b_grad

            layer.update(w_vel, b_vel)
            new_velocities.append([w_vel, b_vel])
        self.velocities = new_velocities

In [47]:
# Setup our parameters
epochs = 100
batch_size = 32
lr = 1e-4

# Run our training loop
sgd = SGDMomentum(lr=lr, alpha=.9)
training_run(epochs, batch_size, sgd, train_data, valid_data)

In this case, SGD with momentum decreases training loss, but increases validation loss.

## Learning Rate

The most important hyperparameter for any optimizer is the learning rate.  The learning rate has a big effect on the performance of a model.  So far, we've used a fixed learning rate that doesn't change during training.

In practice, it's common to use a learning rate scheduler.  A learning rate scheduler changes the learning rate during training.  This helps the model converge, and avoid taking gradient descent steps in the wrong direction.

It's common to alter the learning rate in two ways:

- Set the learning rate low initially as a warmup phase.  This helps to stabilize training, and avoid large gradient steps initially.